# Purpose:
- Adding max_proj to the ops
    - In case where only registraiton was run, there is no max_proj
    - Add using codes from suite2p
- This functionality is added to single_session_suite2p.py

In [29]:
from pathlib import Path
import numpy as np
from suite2p.io.binary import BinaryFile
from suite2p.detection.denoise import pca_denoise
from suite2p.detection import utils
import os
from pprint import pprint
import time

In [2]:
# Load ops
base_dir = Path(r'F:')
mouse = 27
plane = 1
session = '004'
ops_dir = base_dir / f'{mouse:03}' / f'plane_{plane}' / session / 'plane0'
ops_fn = ops_dir / 'ops.npy'
ops = np.load(ops_fn, allow_pickle=True).item()

In [35]:
pprint(list(np.sort(list(ops.keys()))))

['1Preg',
 'Lx',
 'Ly',
 'NRsm',
 'align_by_chan',
 'allow_overlap',
 'anatomical_only',
 'aspect',
 'badframes',
 'baseline',
 'batch_size',
 'bidi_corrected',
 'bidiphase',
 'block_size',
 'bruker',
 'bruker_bidirectional',
 'cellprob_threshold',
 'chan2_thres',
 'classifier_path',
 'combined',
 'connected',
 'corrXY',
 'corrXY1',
 'data_path',
 'date_proc',
 'delete_bin',
 'denoise',
 'diameter',
 'do_bidiphase',
 'do_registration',
 'fast_disk',
 'filelist',
 'flow_threshold',
 'force_refImg',
 'force_sktiff',
 'frames_include',
 'fs',
 'functional_chan',
 'h5list',
 'h5py',
 'h5py_key',
 'high_pass',
 'ignore_flyback',
 'inner_neuropil_radius',
 'input_format',
 'keep_movie_raw',
 'lam_percentile',
 'look_one_level_down',
 'max_iterations',
 'max_overlap',
 'max_proj',
 'maxregshift',
 'maxregshiftNR',
 'meanImg',
 'meanImgE',
 'mesoscan',
 'min_neuropil_pixels',
 'move_bin',
 'multiplane_parallel',
 'nbinned',
 'nblocks',
 'nchannels',
 'neucoeff',
 'neuropil_extract',
 'nframes'

In [30]:
if 'max_proj' in ops.keys():
    print("'max_proj' already in ops.")
else:
    print("Add 'max_proj' to the ops:")

    bin_size = int(max(1, ops['nframes'] // ops['nbinned'], np.round(ops['tau'] * ops['fs'])))
    if not os.path.isfile(ops['reg_file']):
        reg_file = str(Path(ops['save_path']) / 'data.bin')
        if os.path.isfile(reg_file):
            ops['reg_file'] = reg_file
        else:
            raise FileNotFoundError(f'{ops["reg_file"]} or {reg_file} not found')
    
    with BinaryFile(read_filename=ops['reg_file'], Ly=ops['Ly'], Lx=ops['Lx']) as f:
        t0 = time.time()
        mov = f.bin_movie(
            bin_size=bin_size,
            bad_frames=ops.get('badframes'),
            y_range=ops['yrange'],
            x_range=ops['xrange'],
        )
        print('Binned movie [%d,%d,%d] in %0.2f sec.' % (mov.shape[0], mov.shape[1], mov.shape[2], time.time() - t0))
        
    if ops.get('denoise', 1):
        print('PCA denosing...')
        mov = pca_denoise(mov, block_size=[ops['block_size'][0]//2, ops['block_size'][1]//2],
                            n_comps_frac = 0.5)
    else:
        print('No denoising')
    t0 = time.time()
    mov = utils.temporal_high_pass_filter(mov=mov, width=int(ops['high_pass']))
    max_proj = mov.max(axis=0)
    ops['max_proj'] = max_proj
    processing_time = time.time() - t0
    print(f'Added max_proj in {processing_time:.2f} sec.')

# save ops
np.save(ops_fn, ops)

Add 'max_proj' to the ops:
Binned movie [977,385,687] in 27.46 sec.
No denoising
Added max_proj in 1.22 sec.


In [32]:
def add_max_proj(ops_fn):
    ops = np.load(ops_fn, allow_pickle=True).item()
    if 'max_proj' in ops.keys():
        print("'max_proj' already in ops.")
    else:
        print("Add 'max_proj' to the ops:")

        bin_size = int(max(1, ops['nframes'] // ops['nbinned'], np.round(ops['tau'] * ops['fs'])))
        if not os.path.isfile(ops['reg_file']):  # check if data.bin exists in the reg_file
            reg_file = str(Path(ops['save_path']) / 'data.bin')
            if os.path.isfile(reg_file):
                ops['reg_file'] = reg_file
            else:
                raise FileNotFoundError(f'{ops["reg_file"]} or {reg_file} not found')
        
        with BinaryFile(read_filename=ops['reg_file'], Ly=ops['Ly'], Lx=ops['Lx']) as f:
            t0 = time.time()
            mov = f.bin_movie(
                bin_size=bin_size,
                bad_frames=ops.get('badframes'),
                y_range=ops['yrange'],
                x_range=ops['xrange'],
            )
            print('Binned movie [%d,%d,%d] in %0.2f sec.' % (mov.shape[0], mov.shape[1], mov.shape[2], time.time() - t0))
            
        if ops.get('denoise', 1):
            print('PCA denosing...')
            mov = pca_denoise(mov, block_size=[ops['block_size'][0]//2, ops['block_size'][1]//2],
                                n_comps_frac = 0.5)
        else:
            print('No denoising')
        t0 = time.time()
        mov = utils.temporal_high_pass_filter(mov=mov, width=int(ops['high_pass']))
        max_proj = mov.max(axis=0)
        ops['max_proj'] = max_proj
        processing_time = time.time() - t0
        print(f'Added max_proj in {processing_time:.2f} sec.')

    # save ops
    np.save(ops_fn, ops)

In [33]:
# Run for all planes in a session
base_dir = Path(r'F:')
mouse = 27
session = '004'
for plane in range(1,9):
    ops_dir = base_dir / f'{mouse:03}' / f'plane_{plane}' / session / 'plane0'
    ops_fn = ops_dir / 'ops.npy'
    add_max_proj(ops_fn)

'max_proj' already in ops.
Add 'max_proj' to the ops:
Binned movie [984,385,685] in 70.57 sec.
No denoising
Added max_proj in 1.27 sec.
Add 'max_proj' to the ops:
Binned movie [982,381,675] in 71.03 sec.
No denoising
Added max_proj in 1.19 sec.
Add 'max_proj' to the ops:
Binned movie [982,315,609] in 66.06 sec.
No denoising
Added max_proj in 0.91 sec.
Add 'max_proj' to the ops:
Binned movie [902,387,687] in 66.22 sec.
No denoising
Added max_proj in 1.26 sec.
Add 'max_proj' to the ops:
Binned movie [902,387,687] in 61.51 sec.
No denoising
Added max_proj in 1.06 sec.
Add 'max_proj' to the ops:
Binned movie [901,385,685] in 65.90 sec.
No denoising
Added max_proj in 1.08 sec.
Add 'max_proj' to the ops:
Binned movie [902,385,685] in 24.89 sec.
No denoising
Added max_proj in 1.07 sec.
